In [1]:
import pandas as pd
import numpy as np

from model_utils import run_classifier
from sklearn.neural_network import MLPClassifier

In [2]:
# Load datasets
X_train = pd.read_csv("../04_modelling/dataset/X_train.csv")
y_train = pd.read_csv("../04_modelling/dataset/y_train.csv")
X_test = pd.read_csv("../04_modelling/dataset/X_test.csv")
y_test = pd.read_csv("../04_modelling/dataset/y_test.csv")

# Fix the shape of y_train and y_test
y_train = y_train.values.ravel()  # Convert to 1D array
y_test = y_test.values.ravel()    # Convert to 1D array

In [ ]:
mlp = MLPClassifier()

# param_grid_mlp = {'hidden_layer_sizes': [(10,), (50,), (10, 10), (50, 50)],
#              'activation': ['identity', 'logistic', 'tanh', 'relu'],
#              'solver': ['lbfgs', 'sgd', 'adam'],
#              'alpha': np.logspace(-5, 3, 5),
#              'learning_rate': ['constant', 'invscaling','adaptive'],
#              'max_iter': [100, 500, 1000]}

# Updated parameter grid
param_grid_mlp = {
    'hidden_layer_sizes': [(10,), (50,)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['adaptive'],
    'max_iter': [10, 100, 1000]
}

# Run the classifier
best_model = run_classifier(mlp, param_grid_mlp, X_train, y_train, X_test, y_test, "Neural Net")

In [ ]:
mlp_2 = MLPClassifier()

param_grid_mlp_2 = {'hidden_layer_sizes': [(10,), (50,), (10, 10), (50, 50)],
             'activation': ['identity', 'logistic', 'tanh', 'relu'],
             'solver': ['lbfgs', 'sgd', 'adam'],
             'alpha': np.logspace(-5, 3, 5),
             'learning_rate': ['constant', 'invscaling','adaptive'],
             'max_iter': [100, 500, 1000]}

# Run the classifier
best_model = run_classifier(mlp_2, param_grid_mlp_2, X_train, y_train, X_test, y_test, "Neural Net")

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import ConvergenceWarning
import warnings

In [13]:
def run_classifier(clf, param_grid, X_train, y_train, X_test, y_test, title, n_splits=5, random_state=123):
    """
    Perform cross-validation, hyperparameter tuning, and evaluation for a given classifier.
    """
    warnings.filterwarnings("ignore", category=UserWarning)  # Ignore warnings from MLP convergence issues
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    
    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Cross-Validation Setup
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    gs = GridSearchCV(
        clf,
        param_grid=param_grid,
        cv=cv,
        scoring='accuracy',
        return_train_score=True
    )

    # Perform Cross-Validation and Hyperparameter Tuning
    gs.fit(X_train, y_train)

    print(f"\n--- Cross-Validation Results ({title}) ---")
    print("The best parameters are:", gs.best_params_)
    print("Mean cross-validation accuracy: %.2f%%" % (gs.best_score_ * 100))

    # Evaluate Model on Test Set
    print("\n--- Test Results ---")
    y_test_pred = gs.best_estimator_.predict(X_test)

    print('Accuracy: %.2f%%' % (accuracy_score(y_test, y_test_pred) * 100))
    print('Precision: %.2f%%' % (precision_score(y_test, y_test_pred, average='weighted') * 100))
    print('Recall: %.2f%%' % (recall_score(y_test, y_test_pred, average='weighted') * 100))

    return gs.best_estimator_

In [14]:
# Load datasets
X_train = pd.read_csv("../04_modelling/dataset/X_train.csv")
y_train = pd.read_csv("../04_modelling/dataset/y_train.csv")
X_test = pd.read_csv("../04_modelling/dataset/X_test.csv")
y_test = pd.read_csv("../04_modelling/dataset/y_test.csv")

In [15]:
# Fix the shape of y_train and y_test
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [16]:
# Define the MLPClassifier and parameter grid
mlp = MLPClassifier(random_state=123)
param_grid_mlp = {
    'hidden_layer_sizes': [(10,), (50,), (100,)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['adaptive'],
    'max_iter': [200, 500]
}

In [ ]:
# Run the classifier
best_model = run_classifier(mlp, param_grid_mlp, X_train, y_train, X_test, y_test, "Neural Net")